In [ ]:
# from moccasin import setup_notebook

# setup_notebook()

In [2]:
from moccasin.config import get_active_network

active_network = get_active_network()
print(active_network.name)

eth-forked


In [3]:
from boa.contracts.abi.abi_contract import ABIContract
from typing import Tuple
from moccasin.config import get_active_network
import boa

STARTING_ETH_BALANCE = int(1000e18)
STARTING_WETH_BALANCE = int(1e18)
STARTING_USDC_BALANCE = int(100e6)

def _add_eth_balance():
    boa.env.set_balance(boa.env.eoa, STARTING_ETH_BALANCE)

def _add_token_balance(usdc, weth):
    print(f"USDC balance before: {usdc.balanceOf(boa.env.eoa)}")
    weth.deposit(value=STARTING_WETH_BALANCE)
    our_address = boa.env.eoa
    with boa.env.prank(usdc.owner()):
        usdc.updateMasterMinter(our_address)
    usdc.configureMinter(our_address, STARTING_USDC_BALANCE)
    usdc.mint(our_address, STARTING_USDC_BALANCE)
    print(f"USDC balance after: {usdc.balanceOf(boa.env.eoa)}")

def setup_script() -> Tuple[ABIContract, ABIContract, ABIContract, ABIContract]:
    print("Setting up script...")

    # 1. Give ourselve some ETH
    # 2. Give ourselve some USDC and WETH
    active_network = get_active_network()

    usdc = active_network.manifest_named("usdc")
    weth = active_network.manifest_named("weth")

    if active_network.is_local_or_forked_network():
        _add_eth_balance()
        _add_token_balance(usdc, weth)

def moccasin_main():
    setup_script()

moccasin_main()

Setting up script...
USDC balance before: 0
USDC balance after: 100000000


In [4]:
usdc = active_network.manifest_named("usdc")
weth = active_network.manifest_named("weth")

In [5]:
from moccasin.config import get_config

config = get_config()
config.reload()
active_network = config.get_active_network()
aave3_pool_address_provider = active_network.manifest_named("aave3_pool_address_provider")
pool_address = aave3_pool_address_provider.getPool()
print(pool_address)

config.reload()
active_network = config.get_active_network()
pool_contract = active_network.manifest_named("pool", address=pool_address)

0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2


In [6]:
REFERRAL_CODE = 0  # Set to 0 for no referral code

def deposit(pool_contract, token, amount):
    allowed_amount = token.allowance(boa.env.eoa, pool_contract.address)
    if allowed_amount < amount:
        token.approve(pool_contract.address, amount)
    print(f"Depositing {token.name()} into Aave contract {pool_contract.address}...")
    pool_contract.supply(token.address, amount, boa.env.eoa, REFERRAL_CODE)

usdc_balance = usdc.balanceOf(boa.env.eoa)
weth_balance = weth.balanceOf(boa.env.eoa)

if usdc_balance > 0:
    deposit(pool_contract, usdc, usdc_balance)

if weth_balance > 0:
    deposit(pool_contract, weth, weth_balance)

(
        totalCollateralBase,
        totalDebtBase,
        availableBorrowsBase,
        currentLiquidationThreshold,
        ltv,
        healthFactor,
    ) = pool_contract.getUserAccountData(boa.env.eoa)
print(f"""User account data:
        totalCollateralBase: {totalCollateralBase}
        totalDebtBase: {totalDebtBase}
        availableBorrowsBase: {availableBorrowsBase}
        currentLiquidationThreshold: {currentLiquidationThreshold}
        ltv: {ltv}
        healthFactor: {healthFactor}
          """)


Depositing USD Coin into Aave contract 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2...
Depositing Wrapped Ether into Aave contract 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2...
User account data:
        totalCollateralBase: 261680970990
        totalDebtBase: 0
        availableBorrowsBase: 210077483511
        currentLiquidationThreshold: 8280
        ltv: 8028
        healthFactor: 115792089237316195423570985008687907853269984665640564039457584007913129639935
          


In [7]:
# 30% USDC and 70% WETH
config = get_config()
config.reload()
aave_protocol_data_provider = active_network.manifest_named("aave_protocol_data_provider")
a_tokens = aave_protocol_data_provider.getAllATokens()
print(a_tokens)

[getAllATokens(symbol='aEthWETH', tokenAddress=Address('0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8')), getAllATokens(symbol='aEthwstETH', tokenAddress=Address('0x0B925eD163218f6662a35e0f0371Ac234f9E9371')), getAllATokens(symbol='aEthWBTC', tokenAddress=Address('0x5Ee5bf7ae06D1Be5997A1A72006FE6C607eC6DE8')), getAllATokens(symbol='aEthUSDC', tokenAddress=Address('0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c')), getAllATokens(symbol='aEthDAI', tokenAddress=Address('0x018008bfb33d285247A21d44E50697654f754e63')), getAllATokens(symbol='aEthLINK', tokenAddress=Address('0x5E8C8A7243651DB1384C0dDfDbE39761E8e7E51a')), getAllATokens(symbol='aEthAAVE', tokenAddress=Address('0xA700b4eB416Be35b2911fd5Dee80678ff64fF6C9')), getAllATokens(symbol='aEthcbETH', tokenAddress=Address('0x977b6fc5dE62598B08C85AC8Cf2b745874E8b78c')), getAllATokens(symbol='aEthUSDT', tokenAddress=Address('0x23878914EFE38d27C4D67Ab83ed1b93A74D4086a')), getAllATokens(symbol='aEthrETH', tokenAddress=Address('0xCc9EE9483f662091a1de47

In [8]:
for a_token in a_tokens:
    if "WETH" in a_token[0]:
        a_weth = active_network.manifest_named("usdc", address=a_token[1])
    if "USDC" in a_token[0]:
        a_usdc = active_network.manifest_named("usdc", address=a_token[1])

print(f"a_weth: {a_weth}")
print(f"a_usdc: {a_usdc}")

a_weth: <usdc interface at 0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8>
a_usdc: <usdc interface at 0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c>


In [9]:
# Get how much they are worh, to figure out what our portfolio makeup is
a_usdc_balance = a_usdc.balanceOf(boa.env.eoa) # 6 decimals
a_weth_balance = a_weth.balanceOf(boa.env.eoa) # 18 decimals

a_usdc_balance_normalized = a_usdc_balance / (1e6)  # Normalize USDC balance to 6 decimals
a_weth_balance_normalized = a_weth_balance / (1e18)  # Normalize WETH balance to 18 decimals
print(f"a_usdc_balance_normalized: {a_usdc_balance_normalized}")
print(f"a_weth_balance_normalized: {a_weth_balance_normalized}")

a_usdc_balance_normalized: 100.0
a_weth_balance_normalized: 1.0


In [10]:
config.reload()

def get_price(feed_name: str) -> float:
    active_network = get_active_network()
    price_feed = active_network.manifest_named(feed_name)
    price = price_feed.latestAnswer()
    decimals = price_feed.decimals()
    decimals_normalized = 10 ** decimals
    return price / decimals_normalized

usdc_price = get_price("usdc_usd_price_feed")
weth_price = get_price("eth_usd_price_feed")
print(f"USDC price: {usdc_price}")
print(f"WETH price: {weth_price}")

USDC price: 0.99997
WETH price: 2521.78331287


In [11]:
usdc_value = a_usdc_balance_normalized * usdc_price
weth_value = a_weth_balance_normalized * weth_price
total_value = usdc_value + weth_value

target_usdc_value = 0.3
target_weth_value = 0.7

usdc_percent_allocation = usdc_value / total_value
weth_percent_allocation = weth_value / total_value

BUFFER = 0.1

needs_rebalancing = (
    abs(usdc_percent_allocation - target_usdc_value) > BUFFER or abs(weth_percent_allocation - target_weth_value) > BUFFER
)

print(needs_rebalancing)
print(usdc_percent_allocation)
print(weth_percent_allocation)

True
0.038140876834388805
0.9618591231656113


In [12]:
a_weth.approve(pool_contract.address, a_weth.balanceOf(boa.env.eoa))
pool_contract.withdraw(weth.address, a_weth.balanceOf(boa.env.eoa), boa.env.eoa)

def print_token_balances():
    print(f"USDC balance: {usdc.balanceOf(boa.env.eoa)}")  
    print(f"WETH balance: {weth.balanceOf(boa.env.eoa)}")  
    print(f"aUSDC balance: {a_usdc.balanceOf(boa.env.eoa)}")  # Raw USDC balance
    print(f"aWETH balance: {a_weth.balanceOf(boa.env.eoa)}")  # Raw WETH balance

print_token_balances()

USDC balance: 0
WETH balance: 1000000000000000000
aUSDC balance: 100000000
aWETH balance: 0


In [13]:
usdc_data = {"balance": a_usdc_balance_normalized, "price": usdc_price, "contract": usdc}
weth_data = {"balance": a_weth_balance_normalized, "price": weth_price, "contract": weth}
target_allocations = {"usdc": 0.3, "weth": 0.7}

def calculate_rebalancing_trades(
    usdc_data: dict,  # {"balance": float, "price": float, "contract": Contract}
    weth_data: dict,  # {"balance": float, "price": float, "contract": Contract}
    target_allocations: dict[str, float],  # {"usdc": 0.3, "weth": 0.7}
) -> dict[str, dict]:
    """
    Calculate the trades needed to rebalance a portfolio of USDC and WETH.

    Args:
        usdc_data: Dict containing USDC balance, price and contract
        weth_data: Dict containing WETH balance, price and contract
        target_allocations: Dict of token symbol to target allocation (must sum to 1)

    Returns:
        Dict of token symbol to dict containing contract and trade amount:
            {"usdc": {"contract": Contract, "trade": int},
             "weth": {"contract": Contract, "trade": int}}
    """
    # Calculate current values
    usdc_value = usdc_data["balance"] * usdc_data["price"]
    weth_value = weth_data["balance"] * weth_data["price"]
    total_value = usdc_value + weth_value

    # Calculate target values
    target_usdc_value = total_value * target_allocations["usdc"]
    target_weth_value = total_value * target_allocations["weth"]

    # Calculate trades needed in USD
    usdc_trade_usd = target_usdc_value - usdc_value
    weth_trade_usd = target_weth_value - weth_value

    # Convert to token amounts
    return {
        "usdc": {
            "contract": usdc_data["contract"],
            "trade": usdc_trade_usd / usdc_data["price"],
        },
        "weth": {
            "contract": weth_data["contract"],
            "trade": weth_trade_usd / weth_data["price"],
        },
    }

trades = calculate_rebalancing_trades(usdc_data, weth_data, target_allocations)

weth_to_sell = trades["weth"]["trade"]

In [14]:
config.reload()
active_network = config.get_active_network()
uniswap_swap_router = active_network.manifest_named("uniswap_swap_router")

amount_weth = abs(int(weth_to_sell * (10 ** 18)))

weth.approve(uniswap_swap_router.address, amount_weth)

min_out = int((trades["usdc"]["trade"] * (10 ** 6)) * 0.90)  # 1% slippage

print("Let's swap")

uniswap_swap_router.exactInputSingle(
    (
        weth.address,  # tokenIn
        usdc.address,  # tokenOut
        3000,  # fee
        boa.env.eoa,  # recipient
        amount_weth,  # deadline
        min_out,  # amountIn
        0,  # amountOutMinimum
    )
)

print_token_balances()

Let's swap
USDC balance: 683606705
WETH balance: 727757301605876832
aUSDC balance: 100000000
aWETH balance: 0


In [15]:
amountUsdc = usdc.balanceOf(boa.env.eoa)
deposit(pool_contract, usdc, amountUsdc)

amountWeth = weth.balanceOf(boa.env.eoa)
deposit(pool_contract, weth, amountWeth)

print_token_balances()

a_usdc_balance = a_usdc.balanceOf(boa.env.eoa)
a_weth_balance = a_weth.balanceOf(boa.env.eoa)

a_usdc_balance_normalized = a_usdc_balance / (1e6)  # Normalize USDC balance to 6 decimals
a_weth_balance_normalized = a_weth_balance / (1e18)  # Normalize WETH balance to 18 decimals

usdc_value = a_usdc_balance_normalized * usdc_price
weth_value = a_weth_balance_normalized * weth_price

weth_percent_allocation = weth_value / (usdc_value + weth_value)
usdc_percent_allocation = usdc_value / (usdc_value + weth_value)

print(f"Current USDC allocation: {usdc_percent_allocation}")
print(f"Current WETH allocation: {weth_percent_allocation}")

Depositing USD Coin into Aave contract 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2...
Depositing Wrapped Ether into Aave contract 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2...
USDC balance: 0
WETH balance: 0
aUSDC balance: 783606705
aWETH balance: 727757301605876832
Current USDC allocation: 0.299211239979574
Current WETH allocation: 0.7007887600204261
